In [1]:
import pandas as pd
import sys
import dateutil
from tqdm import tqdm

sys.path.append('../..')
pd.set_option('display.max_columns', 500)

In [2]:
LOG = './source/mpd-watcher-log.csv'
df = pd.read_csv(LOG)
df.head(1)

,file,artist,album_artist,title,album,time,type,musicbrainz_albumid,musicbrainz_artistid,musicbrainz_trackid
0,After Forever/2007 - After Forever/01 - Discor...,After Forever,After Forever,Discord,After Forever,2020-07-25 21:59:07,skipped,4e7791ec-ebb2-43d8-b6f6-10d6b89aea7e,6b966946-5274-4e3d-aabb-7563814d805f,1be2a709-9249-4685-998f-e5a58333823e


In [3]:
from api import DBConn
from models import Base, MpdSong, SongListened

DBConn()

In [4]:
DBConn.engine.execute('CREATE SCHEMA IF NOT EXISTS mpd')
tables = []
for name, table in Base.metadata.tables.items():
    if table.schema == 'mpd':
        tables.append(table)
Base.metadata.create_all(DBConn.engine, tables)

In [5]:
records = df.to_dict(orient='records')

with DBConn.get_session() as db:
    for record in tqdm(records):
        if record['type'] == 'skipped':
            continue
        song = db.query(MpdSong).filter_by(file=record['file']).first()
        listened = SongListened(song_id=song.id, time=record['time'])
        db.merge(listened)
    db.commit()

100%|██████████| 296/296 [02:48<00:00,  1.75it/s]
